In [276]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random
import numpy as np

In [277]:
input_path = 'sto/transitive_closure/'
kg = input_path + 'transitive.nt'
dataset = pd.read_csv(kg, delimiter='---', header=None)
dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,<https://w3id.org/i40/sto#ISO_14739-1> <https:...
1,<https://w3id.org/i40/sto#IEC_29190> <https://...
2,<https://w3id.org/i40/sto#ISO_29002> <https://...
3,<https://w3id.org/i40/sto#RFC_7519> <https://w...
4,<https://w3id.org/i40/sto#ISO_15408> <https://...


In [278]:
g = Graph()
g.parse("sto/transitive_closure/transitive.nt", format="nt")

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select distinct ?s where {
            ?s ?p ?o .
    }""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [279]:
entities.shape

(249, 1)

## Select all standard triples given a specific property

In [280]:
count = 0
#prop = 'hasClassification'
prop = 'relatedTo'
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:""" +prop+ """ ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append('<'+str("%s" %row)+'>')
    
    if len(obj)>0:
        lst.o=obj
        lst.s='<'+standard+'>'
        lst.p='<https://w3id.org/i40/sto#'+prop+'>'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1

#list_pd = list_pd.drop_duplicates().reset_index(drop=True)

In [281]:
list_pd.head()

,s,p,o
0,<https://w3id.org/i40/sto#BPMN>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#IEEE_1686>
1,<https://w3id.org/i40/sto#BPMN>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_15704>
2,<https://w3id.org/i40/sto#BPMN>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#DIN_EN_61511>
3,<https://w3id.org/i40/sto#BPMN>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_27017>
4,<https://w3id.org/i40/sto#BPMN>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#IEC_27036>


In [282]:
list_pd.shape

(22969, 3)

In [322]:
import numpy as np
from sklearn.model_selection import KFold
test_set = []
training_set = []
X = list_pd
kf = KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

print(kf)  

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(test_index))
    test_set.append(list_pd.iloc[test_index])
    training_set.append(list_pd.iloc[train_index])

KFold(n_splits=5, random_state=0, shuffle=True)
TRAIN: [    0     1     2 ... 22964 22965 22967] TEST: [    7    12    16 ... 22957 22966 22968]
4594
TRAIN: [    0     2     3 ... 22964 22966 22968] TEST: [    1     4     8 ... 22959 22965 22967]
4594
TRAIN: [    1     4     7 ... 22966 22967 22968] TEST: [    0     2     3 ... 22947 22951 22964]
4594
TRAIN: [    0     1     2 ... 22966 22967 22968] TEST: [   11    15    21 ... 22960 22961 22963]
4594
TRAIN: [    0     1     2 ... 22966 22967 22968] TEST: [   10    13    19 ... 22946 22948 22962]
4593


In [323]:
i=0
for test in test_set:
    test['dot'] = '.'
    test_set_nt=pd.DataFrame(test['s'] + ' '+ test['p']+ ' '+ test['o']+ ' '+ test['dot'])
    test_set_nt.to_csv(input_path+'test_set_relatedTo'+str(i)+'.nt', index=None, header=None)
    i+=1
i=0
for training in training_set:
    training['dot'] = '.'
    training_set_nt=pd.DataFrame(training['s'] + ' '+ training['p']+ ' '+ training['o']+ ' '+ training['dot'])
    training_set_nt.to_csv(input_path+'training_set_relatedTo'+str(i)+'nt', index=None, header=None)
    i+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Create the test set [Select 30% of triple randomly] 

In [283]:
n_select= round(1*list_pd.shape[0]/10)
print(n_select)
index = list_pd.index.tolist()
#sampling = random.choices(index, k=n_select)
sampling = random.sample(index, k=n_select)

2297


In [284]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

2297


,s,p,o


In [ ]:
#test_set.sort_values(by='s', ascending=False)

In [285]:
test_set['dot'] = '.'

In [286]:
test_set_nt=pd.DataFrame(test_set['s'] + ' '+ test_set['p']+ ' '+ test_set['o']+ ' '+ test_set['dot'])
test_set_nt.head(20)

,0
0,<https://w3id.org/i40/sto#ISO_27019> <https://...
1,<https://w3id.org/i40/sto#IEC_27036> <https://...
2,<https://w3id.org/i40/sto#ISA-95> <https://w3i...
3,<https://w3id.org/i40/sto#IEC_24760> <https://...
4,<https://w3id.org/i40/sto#IEC_61850_P7_S2_E2> ...
5,<https://w3id.org/i40/sto#ebXML> <https://w3id...
6,<https://w3id.org/i40/sto#DIN_EN_50129> <https...
7,<https://w3id.org/i40/sto#ISO_15926> <https://...
8,<https://w3id.org/i40/sto#ISO_27036> <https://...
9,<https://w3id.org/i40/sto#DIN_EN_61511> <https...


## Create training test [difference bettwen dataset and test set]

In [287]:
training_set = dataset[~dataset.apply(tuple,1).isin(test_set_nt.apply(tuple,1))]
training_set.shape[0]

20672

In [288]:
dataset.shape[0]-training_set.shape[0]

2297

## Save training and test set

In [289]:
training_set.to_csv(input_path+'training_set_relatedTo.nt', index=None, header=None)
test_set_nt.to_csv(input_path+'test_set_relatedTo.nt', index=None, header=None)

## Cheking the training set created

In [290]:
g = Graph()
g.parse("sto/training_set_relatedTo.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select distinct ?s where {
            ?s ?p ?o .
    }""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [291]:
entities

,s
0,https://w3id.org/i40/sto#IEC_61804
1,https://w3id.org/i40/sto#IEC_61850
2,https://w3id.org/i40/sto#IEC_62443
3,https://w3id.org/i40/sto#IEC_61987_X
4,https://w3id.org/i40/sto#IEC_61804_P3
5,https://w3id.org/i40/sto#IEC_24760
6,https://w3id.org/i40/sto#IEC_29115
7,https://w3id.org/i40/sto#IEC_61360
8,https://w3id.org/i40/sto#ISO_27031
9,https://w3id.org/i40/sto#ISO_29190
